## The Graph Notebook 

Looking at how to derive price for tokens and gas fees by pool.  Pool example can be found [here](https://app.uniswap.org/explore/pools/ethereum/0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640)

In [128]:
import requests
import time 
import datetime
import os
import pytz
import pandas as pd
import numpy as np
from web3 import Web3

from pandas import json_normalize
 

In [129]:
# change the active path to the parent directory 
if True: 
    print("Moving active path to parent directory")
    os.chdir('..')
    print(os.getcwd())

Moving active path to parent directory
/Users/das/DATASCI210


In [130]:
import src.arbutils as arbutils

In [131]:
API_KEY =  os.getenv('GRAPH_API_KEY')
API_KEY2 =  os.getenv('ETHERSCAN_API_KEY')

if not API_KEY:
    print("No GRAPH_API_KEY found")
else:
    print("Found GRAPH API Key!")


if not API_KEY2:
    print("No ETHERSCAN_API_KEY found")
else:
    print("Found ETHERSCAN API Key!")

Found GRAPH API Key!
Found ETHERSCAN API Key!


In [132]:
POOL0_ADDRESS="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" # USDC / WETH (0.05%) 
POOL0_TXN_FEE = 0.0005
POOL1_ADDRESS="0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" # USDC / WETH (0.3%)
POOl1_TXN_FEE = 0.003

GWEI_SCALER = 1e9

In [133]:
def fetch_data_with_pagination(thegraph_api_key, pool_address, old_date, new_date, data_path=None, checkpoint_file='checkpoint.json',batch_size=1000):

    query_template = """
    {
        swaps(
            first: %s,
            where: {
                pool: "%s",
                timestamp_gt: "%s",
                timestamp_lt: "%s"
            },
            orderBy: timestamp
        ) {
            id
            timestamp
            sender
            recipient
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            pool {
                id
                token0 {
                    id
                    symbol
                    name
                }
                token1 {
                    id
                    symbol
                    name
                }
            }
            transaction {
                id
                blockNumber
                gasUsed
                gasPrice
            }
        }
    }
    """
    newest_id = int(new_date.timestamp())

    if data_path and os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
        oldest_id = int(checkpoint['last_timestamp'])
        batch_num = checkpoint['batch_num']
        all_data = checkpoint.get('accumulated_data', [])
    
    else:
        oldest_id = int(old_date.timestamp())
        batch_num = 0
        all_data = []
    

            
    #
    #  Discussion of loop:
    #  The goal of the loop is to extract all relevant transactions into 
    #  all_data list.  The bounds are oldest_id to newest_id.  These are 
    #  epoch timestamps (i.e. seconds based).  When a batch of 'batch_size'
    #  or less transactions is returned from the response, the oldest_id
    #  is changed in two ways (1) its updated to the newest timestamp in the
    #  "new_data" batch of responses and (2) the timestamp found is decremented
    #  by 1 second for a specific race condition (see next paragraph).  The oldest_id
    #  is thus approaching newest_id until oldest_id == newest_id.  The program breaks 
    #  at this point.
    #
    #  Race condition: it is observed that there are multiple transactions that occur
    #  within the same timestamp (i.e. multiple transaction can occur in the same block
    #  from within the same pool).  Because of this, when we use "first: 1000" in the GraphQL
    #  if there is a case where multiple transactions occur with the same timestamp and just
    #  happens to occur at the end of the list of 1000, the transactions afterward may be 
    #  excluded if care is not taken for this case in managing timestmaps.  The decrement by 1
    #  is intended to pick up where the request left off assuming the last timestamp could have
    #  had multiple transactions.  
    #
    while True:    
        query = query_template % (batch_size, pool_address, oldest_id, newest_id)

        try:
            response = requests.post(
                f'https://gateway.thegraph.com/api/{thegraph_api_key}/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV',
                json={'query': query}
            )  
        except Exception as e:
            print(f"Post error occurred: {e}")
            break
    
        data = response.json()
        if 'errors' in data:
            print("GraphQL query error:", data['errors'])
            break
            
        new_data = data['data']['swaps'] if 'swaps' in data['data'] else []
        if not new_data:
            print("No data found in the fetch...continuing?")
            continue
                            
        # Save the batch immediately to CSV
        batch_df = json_normalize(new_data)
        batch_df['datetime'] = batch_df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x),tz=pytz.UTC))

        # Update checkpoint
        if (int(new_data[0]['timestamp']) - int(new_data[-1]['timestamp'])) != 0: 
            oldest_id = int(new_data[-1]['timestamp'])-1
        else:
            #print("Timestamps are the same!")
            # Exit Loop
            break

        # add batch to all_data
        all_data.extend(new_data)
        
        print(f"{batch_num}: [{oldest_id}-{newest_id}]found {len(new_data)} swaps from {new_data[0]['timestamp']} to {new_data[-1]['timestamp']}")
        batch_num += 1

        if data_path:
            batch_df.to_csv(f'{data_path}/{pool_address}/pool_id_{pool_address}_swap_batch_{batch_num}.csv')
            checkpoint = {
                'last_timestamp': oldest_id,
                'batch_num': batch_num
            }
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f)
    
    print(f"Found {len(all_data)} swaps from {all_data[0]['timestamp']} to {all_data[-1]['timestamp']}")

    return all_data

In [134]:
# Creates datetime objects.  
#                                YYYY  MM  DD  HH  MM  SS
old_date = datetime.datetime(*(2025,  1, 14,  0,  0,  0), tzinfo=pytz.UTC)
new_date = datetime.datetime(*(2025,  1, 15,  0,  0,  0), tzinfo=pytz.UTC)
#new_date = None
#old_date = None

if new_date == None or old_date == None: 
    # Create timestamps based on the latest timestamp
    new_date = datetime.datetime.now(pytz.UTC)
    old_date = new_date - datetime.timedelta(hours=36)

    full_start = time.time()
    
p0 = fetch_data_with_pagination(
    thegraph_api_key=API_KEY,
    pool_address=POOL0_ADDRESS,
    old_date=old_date,
    new_date=new_date)

#   valid_columns = ['transactionHash', 'datetime', 'timeStamp', 'sqrtPriceX96',
#            'blockNumber', 'gasPrice', 'gasUsed', 'tick', 'amount0', 'amount1',
#            'liquidity']

#    valid_columns = ['amount0', 'amount1', 'amountUSD', 'id', 'recipient', 'sender',
#                        'sqrtPriceX96', 'tick', 'timestamp', 'pool.id', 'pool.token0.id',
#                        'pool.token0.name', 'pool.token0.symbol', 'pool.token1.id',
#                        'pool.token1.name', 'pool.token1.symbol', 'transaction.blockNumber',
#                        'transaction.gasPrice', 'transaction.gasUsed', 'transaction.id',
#                        'time']

# Final processing of all data
if p0:
    p0 = json_normalize(p0)
    print(f"Swaps Found (Prior to drop_duplicates): {p0.shape}")
    p0 = p0.drop_duplicates()
    print(f"Swaps Found (After to drop_duplicates): {p0.shape}")

else:
    f"Error Processing final data: {len(p0)} total swaps"

p0['transactionHash'] = p0['transaction.id']
p0['datetime'] = p0['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x),tz=pytz.UTC))
p0['timeStamp'] = p0['timestamp'].astype('int64')
p0['sqrtPriceX96'] = p0['sqrtPriceX96'].astype('float')
p0['blockNumber'] = p0['transaction.blockNumber'].astype('int32')
p0['gasPrice'] = p0['transaction.gasPrice'].astype('float')
p0['gasUsed'] = p0['transaction.gasUsed'].astype('float')
p0['tick'] = p0['tick'].astype('float')
p0['amount0'] = p0['amount0'].astype('float')
p0['amount1'] = p0['amount1'].astype('float')
p0['liquidity'] = -1.0  #not implemented
p0 = p0[['transactionHash', 'datetime', 'timeStamp', 'sqrtPriceX96',
                'blockNumber', 'gasPrice', 'gasUsed', 'tick', 'amount0', 'amount1',
                'liquidity']]

0: [1736830234-1736899200]found 1000 swaps from 1736812823 to 1736830235
1: [1736848918-1736899200]found 1000 swaps from 1736830235 to 1736848919
2: [1736863630-1736899200]found 1000 swaps from 1736848919 to 1736863631
3: [1736879158-1736899200]found 1000 swaps from 1736863631 to 1736879159
4: [1736898130-1736899200]found 1000 swaps from 1736879159 to 1736898131
5: [1736899198-1736899200]found 42 swaps from 1736898131 to 1736899199
Found 5042 swaps from 1736812823 to 1736899199
Swaps Found (Prior to drop_duplicates): (5042, 20)
Swaps Found (After to drop_duplicates): (5034, 20)


In [135]:
p0.iloc[:10]

,transactionHash,datetime,timeStamp,sqrtPriceX96,blockNumber,gasPrice,gasUsed,tick,amount0,amount1,liquidity
0,0x0a4f0781ccb3c2b41c3d175f349931f0380da3821b03...,2025-01-14 00:00:23+00:00,1736812823,1.414924e+33,21619006,4.952162e+09,0.0,195814.0,-126.920283,0.040500,-1.0
1,0xc785ef7d444cad03b8d60abd68eb32570911df1dd30d...,2025-01-14 00:00:23+00:00,1736812823,1.414923e+33,21619006,3.922162e+09,0.0,195814.0,1406.797670,-0.448460,-1.0
2,0x783b5daadbad8658d8304ed7b072b53ac8e3bc05f5cd...,2025-01-14 00:00:47+00:00,1736812847,1.414905e+33,21619008,3.051628e+09,0.0,195814.0,2995.399428,-0.954858,-1.0
3,0x576ea450885b24aad3a5d918b88c0f5409c20fbd5654...,2025-01-14 00:01:11+00:00,1736812871,1.414771e+33,21619010,4.046162e+09,0.0,195812.0,20774.014155,-6.621513,-1.0
4,0xb018854dece107755eb20467e03e36143ce8efebd364...,2025-01-14 00:02:35+00:00,1736812955,1.414771e+33,21619017,4.215761e+09,0.0,195812.0,-62.141680,0.019825,-1.0
5,0xd4e432cabdcf7f22b6648a314bcd3906d710f9fdf285...,2025-01-14 00:02:59+00:00,1736812979,1.415153e+33,21619019,1.394557e+10,0.0,195818.0,-59206.642476,18.893747,-1.0
6,0xba788aee9f4f151246814ac3eca5bc6c612152652567...,2025-01-14 00:03:23+00:00,1736813003,1.415156e+33,21619021,5.363725e+09,0.0,195818.0,-438.731743,0.140044,-1.0
7,0xf9b33c0ef48f03f87a5efb68902f4481bf061e24eef9...,2025-01-14 00:03:35+00:00,1736813015,1.414848e+33,21619022,3.180501e+10,0.0,195813.0,47734.375000,-15.218401,-1.0
8,0xd83d9678e5771346df5c85ef9fbce9035f5fed980b80...,2025-01-14 00:03:59+00:00,1736813039,1.414829e+33,21619024,5.650864e+09,0.0,195813.0,2951.400302,-0.940731,-1.0
9,0x93b3f204df2b85a85e41176037777372b1da63367c0e...,2025-01-14 00:04:11+00:00,1736813051,1.414937e+33,21619025,3.762239e+09,0.0,195815.0,-16732.549481,5.339016,-1.0


Now that I have information at the transaction level.  I would like to parse specific transactions to extract the gasUsed value which is the only value missing for our current model.   Below we will fetch token transfers based on the blocks that we have in the data frame above.

In [136]:
def fetch_tokentx_data(etherscan_api_key, pool_address, start_block=0, end_block=99999999):

    base_url = 'https://api.etherscan.io/api'

    params = {
        'module': 'account',
        'action': 'tokentx',
        'address': pool_address,
        'startblock': start_block,
        'endblock': end_block,
        'sort': 'desc',
        'apikey': etherscan_api_key
    }
    
    
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        #st.error(f"API request failed with status code {response.status_code}")
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    if data['status'] != '1':
        #st.error(f"API returned an error: {data['result']}")
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])
    
    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    df.sort_values(by='timeStamp')
    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': np.int32(row['blockNumber']),
                'timeStamp': int(row['timeStamp']),
                'transactionHash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': float(row['gas']),
                'gasPrice': float(row['gasPrice']),
                'gasUsed': float(row['gasUsed']),
                'cumulativeGasUsed': float(row['cumulativeGasUsed']),
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = float(row['value'])
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = float(row['value'])
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']

    final_df = pd.DataFrame.from_dict(consolidated_data, orient='index').reset_index(drop=True)

    return final_df.sort_values(by='timeStamp')

Here we can access the list of blocks that we need to get the gasUsed values for...

In [137]:
def get_chunks(start_block_num, end_block_num):
    chunks = []
    while start_block_num < end_block_num:
        s = start_block_num
        e = start_block_num+2500
        if e > end_block_num: 
            e = end_block_num
        start_block_num = e+1
        chunks.append((s,e))
    return chunks

In [138]:
# Example usage
p0_startblock = int(p0['blockNumber'].iloc[0])
p0_endblock = int(p0['blockNumber'].iloc[-1])
print(f"Start Block: {p0_startblock}")
print(f"End Block: {p0_endblock}")
print(f"Total Block Search Space {p0_endblock-p0_startblock}")
print(f"Break up into 2500 block chunks...")

block_chunks = get_chunks(p0_startblock, p0_endblock)
print(block_chunks)

Start Block: 21619006
End Block: 21626161
Total Block Search Space 7155
Break up into 2500 block chunks...
[(21619006, 21621506), (21621507, 21624007), (21624008, 21626161)]


In [139]:
import time
start_time = time.time()
p0_tokentx = pd.DataFrame()
for startblock, endblock in block_chunks:
    p0_tokentx = pd.concat([p0_tokentx, fetch_tokentx_data(API_KEY2, POOL0_ADDRESS, start_block=startblock, end_block=endblock)])
    print(p0_tokentx.shape)
    time.sleep(0.01)
end_time = time.time()
print(f"Time to Request: {end_time - start_time}")

(1606, 14)
(3638, 14)
(5069, 14)
Time to Request: 3.032917022705078


Use the fetch_tokentx_data method to fetch the token transfers in that block range from Etherscan...

In [140]:
print(p0.shape)
print(p0_tokentx.shape)

(5034, 11)
(5069, 14)


In [141]:
p0_tokentx.sort_values(by='blockNumber').head()

,blockNumber,timeStamp,transactionHash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations
1605,21619006,1736812823,0xc785ef7d444cad03b8d60abd68eb32570911df1dd30d...,0xf2614a233c7c3e7f08b1f887ba133a13f1eb2c55,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,4.484601e+17,1.406798e+09,Wrapped Ether,USDC,256995.0,3.922162e+09,190978.0,7834749.0,16076
1604,21619006,1736812823,0x0a4f0781ccb3c2b41c3d175f349931f0380da3821b03...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,4.050000e+16,1.269203e+08,Wrapped Ether,USDC,440375.0,4.952162e+09,307867.0,14416301.0,16076
1603,21619007,1736812835,0x3db31054ffa754ab45e464066f1a927f8334fada0a4b...,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,5.889858e+16,3.000000e+07,Wrapped Ether,USDC,489707.0,4.029468e+09,467222.0,7574374.0,16075
1602,21619008,1736812847,0x783b5daadbad8658d8304ed7b072b53ac8e3bc05f5cd...,0xf2614a233c7c3e7f08b1f887ba133a13f1eb2c55,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,9.548577e+17,2.995399e+09,Wrapped Ether,USDC,423017.0,3.051628e+09,283052.0,13736343.0,16074
1601,21619010,1736812871,0x576ea450885b24aad3a5d918b88c0f5409c20fbd5654...,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,6.621513e+18,2.077401e+10,Wrapped Ether,USDC,226476.0,4.046162e+09,113269.0,10851137.0,16072


Below we can see that the token transfer has slightly more blocks available to it. this is because we really only care about swaps in this context and token transfer includes other types of transactions.

In [142]:
print(f"p0 number of unique blocks: {p0.blockNumber.unique().shape[0]}, p0 tx found: {p0.shape[0]}")
print(f"p0_tokentx number of unique blocks: {p0_tokentx.blockNumber.unique().shape[0]}, p0_tokentx tx found: {p0_tokentx.shape[0]}")

p0 number of unique blocks: 3415, p0 tx found: 5034
p0_tokentx number of unique blocks: 3474, p0_tokentx tx found: 5069


In order to merge the data frames we take only what we need from p0_tokentx (i.e. gasUsed). So we drop 'gasUsed' from p0 and we only merge 'gasUsed' from p0_token.  We can see that we end up with the same number of blocks and we can see that the p0_new has now integrated the missing column.

In [143]:
p0.shape

(5034, 11)

In [144]:
#p1_new = pd.merge(p1.drop(labels=['gasUsed'],axis=1), p1_tokentx[['timeStamp','blockNumber','gasUsed']], on=['timeStamp','blockNumber'], how='left')
p0_new = pd.merge(p0.drop(labels=['gasUsed'],axis=1), p0_tokentx[['timeStamp','blockNumber','transactionHash','gasUsed']], on=['timeStamp','blockNumber','transactionHash'], how='left')
print(p0_new.shape)
print(p0_new.blockNumber.unique().shape)
display(p0_new.columns)
p0_new.head()

(5034, 11)
(3415,)


Index(['transactionHash', 'datetime', 'timeStamp', 'sqrtPriceX96',
       'blockNumber', 'gasPrice', 'tick', 'amount0', 'amount1', 'liquidity',
       'gasUsed'],
      dtype='object')

,transactionHash,datetime,timeStamp,sqrtPriceX96,blockNumber,gasPrice,tick,amount0,amount1,liquidity,gasUsed
0,0x0a4f0781ccb3c2b41c3d175f349931f0380da3821b03...,2025-01-14 00:00:23+00:00,1736812823,1.414924e+33,21619006,4.952162e+09,195814.0,-126.920283,0.040500,-1.0,307867.0
1,0xc785ef7d444cad03b8d60abd68eb32570911df1dd30d...,2025-01-14 00:00:23+00:00,1736812823,1.414923e+33,21619006,3.922162e+09,195814.0,1406.797670,-0.448460,-1.0,190978.0
2,0x783b5daadbad8658d8304ed7b072b53ac8e3bc05f5cd...,2025-01-14 00:00:47+00:00,1736812847,1.414905e+33,21619008,3.051628e+09,195814.0,2995.399428,-0.954858,-1.0,283052.0
3,0x576ea450885b24aad3a5d918b88c0f5409c20fbd5654...,2025-01-14 00:01:11+00:00,1736812871,1.414771e+33,21619010,4.046162e+09,195812.0,20774.014155,-6.621513,-1.0,113269.0
4,0xb018854dece107755eb20467e03e36143ce8efebd364...,2025-01-14 00:02:35+00:00,1736812955,1.414771e+33,21619017,4.215761e+09,195812.0,-62.141680,0.019825,-1.0,373853.0


In [145]:
block_cnt = 0
for block in list(p0['blockNumber'].unique()):
  if block in list(p0_new.blockNumber.unique()):
      block_cnt += 1
print(f"{block_cnt}")

3415


**Get price within the Pool in ETH/USDC**

To derive the price for the pool in ETH/USDC, you must use the sqrtPriceX96 value, which is the pool price immediately after the transaction takes place (including slippage).  You can see below that there is almost always a descrepency, but its not always enough to over come transaction and gas fees (see below).

In [146]:
# row to pick for the swap...used just for the example.
pool0_price_in_USDC_per_ETH  = ((p0_new["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1
#pool1_price_in_USDC_per_ETH  = ((p1_new["sqrtPriceX96"].iloc[0] / 2**96)**2 / 1e12) **-1

print(f"Pool 0 Price in USDC per ETH (at Tx: 0x...{p0_new['transactionHash'].iloc[0][-4:]}): ${pool0_price_in_USDC_per_ETH:.2f}")
#print(f"Pool 1 Price in USDC per ETH (at Tx: 0x...{p1_new['transactionHash'].iloc[0][-4:]}): ${pool1_price_in_USDC_per_ETH:.2f}")
#print(f"Difference in price: ${pool1_price_in_USDC_per_ETH-pool0_price_in_USDC_per_ETH:.2f}")

Pool 0 Price in USDC per ETH (at Tx: 0x...2dd5): $3135.40


**Get gas fees in ETH**

Gas fees for a transaction include all the 'work' done. There is a rate of fee per unit of work (i.e. gasPrice) and then there is the work done (i.e. gasUsed).  gasPrice and gasUsed is in gwei which is 1e9 of an ETH.  so to convert to eth, each value needs to be converted with the 1e9 scaling.

In [147]:
print(f"Txn ID: {p0_new['transactionHash'].iloc[0]}")
gas_price_eth_tokens_per_unit = int(p0_new['gasPrice'].iloc[0])/GWEI_SCALER
gas_used_units = int(p0_new['gasUsed'].iloc[0]) / GWEI_SCALER
gas_fees_eth_tokens  = gas_price_eth_tokens_per_unit* gas_used_units
gas_fees_usdc_tokens = pool0_price_in_USDC_per_ETH * gas_fees_eth_tokens 
print(f"Gas Price in ETH per unit: {gas_price_eth_tokens_per_unit}")
print(f"Gas Used in GWEI units for Uniswap Transaction: {gas_used_units}")
print(f"Gas fees for this Transaction in ETH: {gas_fees_eth_tokens:.5f} (ETH)")
print(f"Gas fees for this Transaction in USDC: ${gas_fees_usdc_tokens:.2f} (USDC)")

Txn ID: 0x0a4f0781ccb3c2b41c3d175f349931f0380da3821b03e88c0d9cf392d2ef2dd5
Gas Price in ETH per unit: 4.952162373
Gas Used in GWEI units for Uniswap Transaction: 0.000307867
Gas fees for this Transaction in ETH: 0.00152 (ETH)
Gas fees for this Transaction in USDC: $4.78 (USDC)
